# Análise Exploratória dos Dados de Prefeitos (2k-200k habitantes)

Este notebook apresenta uma análise exploratória do arquivo `dados_unificados_prefeitos_200k.csv`, que consolida as informações de resultados eleitorais, valores de emendas PIX e indicadores socioeconômicos dos municípios.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm # estimação de modelos
from scipy import stats # estatística chi2
from statsmodels.iolib.summary2 import summary_col # comparação entre modelos
from scipy.stats import gaussian_kde # inserção de KDEs em gráficos
from matplotlib.gridspec import GridSpec # plotagem de gráficos separados
import time # definição do intervalo de tempo entre gráficos com animação
import imageio # para geração de figura GIF
from tqdm import tqdm # adiciona um indicador de progresso do código

In [ ]:
sns.set(style='whitegrid')
pd.options.display.float_format = '{:.2f}'.format

path = '../data/dados_unificados_prefeitos_200k.csv'
df = pd.read_csv(path)

In [ ]:
df.head()

## Visão Geral

A primeira etapa consiste em verificar o formato da base e o tipo das variáveis.

### Filtragem de casos sem emendas ou com votação unânime e partidos com menos de 10 candidatos
As análises a seguir desconsideram municípios que receberam soma zero de Emendas PIX e onde apenas um candidato concorreu (100% dos votos válidos).

In [ ]:
# Contar e printar 0s da coluna 'emendas_pix_per_capita_partido_prefeito_eleito'
# zero_count = (df['emendas_pix_per_capita_partido_prefeito_eleito'] == 0).sum()
# print(f'Número de zeros na coluna "emendas_pix_per_capita_partido_prefeito_eleito": {zero_count}')

# Contar e printar 1s da coluna 'porcentagem_votos_validos_2024'
one_count = (df['porcentagem_votos_validos_2024'] == 1).sum()
print(f'Número de uns na coluna "porcentagem_votos_validos_2024": {one_count}')
    

In [ ]:
# Remover 0s da coluna 'emendas_pix_per_capita_partido_prefeito_eleito'
# df = df[df['emendas_pix_per_capita_partido_prefeito_eleito'] > 0]

# Remover 1s da coluna 'porcentual_votos_partido_prefeito_eleito'
df = df[df['porcentagem_votos_validos_2024'] < 1]

# Contar e printar onde a soma de municipios de determinado partido é menor que 10
parties = df['sigla_partido_prefeito_eleito'].value_counts()
low_count_parties = parties[parties < 10]
print("Partidos com menos de 10 municípios:")
for party, count in low_count_parties.items():
    print(f'{party}: {count} municípios')
    
# Remover partidos com menos de 10 municípios
df = df[df['sigla_partido_prefeito_eleito'].isin(parties[parties >= 10].index)]

# Remover linhas com valores NaN
df = df.dropna(subset=['emendas_pix_per_capita_partido_prefeito_eleito', 'porcentagem_votos_validos_2024', 'sigla_partido_prefeito_eleito'])

# Remover linhas com valores NaN
df.shape, df.dtypes

In [ ]:
# Remover valores extremos da coluna 'emendas_pix_per_capita_partido_prefeito_eleito'
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

df = remove_outliers(df, 'emendas_pix_per_capita_partido_prefeito_eleito')

In [ ]:
# Atribuição de categorias para as variáveis 'prefeito_eleito_2024', 'sigla_partido_prefeito_eleito', 'municipio', and 'sigla_municipio'
df['prefeito_eleito_2024'] = df['prefeito_eleito_2024'].astype('category')
df['sigla_partido_prefeito_eleito'] = df['sigla_partido_prefeito_eleito'].astype('category')
df['municipio'] = df['municipio'].astype('category')
df['sigla_municipio'] = df['sigla_municipio'].astype('category')

In [ ]:
df.describe().round(2)

## Estudo sobre o desbalanceamento dos dados por partido

In [ ]:
# Group by 'sigla_partido_prefeito_eleito'
df.groupby('sigla_partido_prefeito_eleito')['municipio'].count().reset_index()

desempenho_medio = df.groupby('sigla_partido_prefeito_eleito')['porcentagem_votos_validos_2024'].mean().reset_index()
desempenho_medio

## Desempenho médio eleição por partido

In [ ]:
plt.figure(figsize=(36,10))
plt.plot(desempenho_medio['sigla_partido_prefeito_eleito'], desempenho_medio['porcentagem_votos_validos_2024'],
         linewidth=5, color='indigo')
plt.scatter(df['sigla_partido_prefeito_eleito'], df['porcentagem_votos_validos_2024'],
            alpha=0.5, color='orange', s = 150)
plt.xlabel('Partido $j$ (nível 2)', fontsize=20)
plt.ylabel('Desempenho eleitoral', fontsize=20)
plt.xticks(desempenho_medio.sigla_partido_prefeito_eleito, fontsize=17)
plt.yticks(fontsize=17)
plt.show()

## Boxplot da variável dependente ('desempenho')

In [ ]:
plt.figure(figsize=(15,15))
sns.boxplot(data=df, y='porcentagem_votos_validos_2024',
            linewidth=2, orient='v', color='deepskyblue')
sns.stripplot(data=df, y='porcentagem_votos_validos_2024',
              color='darkorange', jitter=0.1, size=12, alpha=0.5)
plt.ylabel('Desempenho eleitoral', fontsize=20)
plt.yticks(fontsize=17)
plt.show()

## Kernel density estimation (KDE) - função densidade de probabilidade

In [ ]:
#da variável dependente ('porcentagem_votos_validos_2024'), com histograma
plt.figure(figsize=(15,10))
sns.histplot(data=df['porcentagem_votos_validos_2024'], kde=True,
             bins=30, color='deepskyblue')
plt.xlabel('Desempenho eleitoral', fontsize=20)
plt.ylabel('Contagem', fontsize=20)
plt.tick_params(axis='y', labelsize=17)
plt.tick_params(axis='x', labelsize=17)
plt.show()

## Boxplot da variável dependente ('desempenho') por sigla_partido_prefeito_eleito

In [ ]:
plt.figure(figsize=(50,14))
sns.boxplot(data=df, x='sigla_partido_prefeito_eleito', y='porcentagem_votos_validos_2024',
            linewidth=2, orient='v', palette='viridis')
sns.stripplot(data=df, x='sigla_partido_prefeito_eleito', y='porcentagem_votos_validos_2024',
              palette='viridis', jitter=0.2, size=8, alpha=0.5)
plt.ylabel('Desempenho eleitoral', fontsize=20)
plt.xlabel('Partido $j$ (nível 2)', fontsize=20)
plt.tick_params(axis='y', labelsize=17)
plt.tick_params(axis='x', labelsize=17)
plt.show()


## Kernel density estimation (KDE) - função densidade de probabilidade

In [ ]:
#da variável dependente ('desempenho') por sigla_partido_prefeito_eleito

partidos = df['sigla_partido_prefeito_eleito'].unique()
colors = sns.color_palette('viridis', len(partidos))

plt.figure(figsize=(15, 10))
g = sns.pairplot(df[['sigla_partido_prefeito_eleito', 'porcentagem_votos_validos_2024']], hue='sigla_partido_prefeito_eleito',
                 height=8,
                 aspect=1.5, palette=colors)
g._legend.remove()
g.set(xlabel=None)
g.set(ylabel=None)
g.tick_params(axis='both', which='major', labelsize=15)

# Gera a legenda com cores e rótulos das municipios
legend_elements = [plt.Line2D([0], [0], marker='o', color='w',
                              markerfacecolor=color,
                              markersize=10, label=sigla_partido_prefeito_eleito)
                   for sigla_partido_prefeito_eleito, color in zip(partidos, colors)]
plt.legend(handles=legend_elements, title='Partido', fontsize=14,
           title_fontsize=18)

# Adiciona os rótulos diretamente na figura
plt.gcf().text(0.5, -0.01, 'Desempenho eleitoral', ha='center', fontsize=20)
plt.gcf().text(-0.01, 0.5, 'Frequência', va='center', rotation='vertical',
               fontsize=20)
plt.show()



## Kernel density estimation (KDE) - função densidade de probabilidade**

In [ ]:

#da variável dependente ('porcentagem_votos_validos_2024'), com histograma e por sigla_partido_prefeito_eleito separadamente
#(função 'GridSpec' do pacote 'matplotlib.gridspec')

partidos = df['sigla_partido_prefeito_eleito'].unique()

fig = plt.figure(figsize=(28, 20))
gs = GridSpec(len(partidos) // 2 + 1, 2, figure=fig)

for i, sigla_partido_prefeito_eleito in enumerate(partidos):
    ax = fig.add_subplot(gs[i])

    # Subset dos dados por sigla_partido_prefeito_eleito
    df_escola = df[df['sigla_partido_prefeito_eleito'] == sigla_partido_prefeito_eleito]

    # Densidade dos dados
    densidade = gaussian_kde(df_escola['porcentagem_votos_validos_2024'])
    x_vals = np.linspace(min(df_escola['porcentagem_votos_validos_2024']),
                         max(df_escola['porcentagem_votos_validos_2024']), len(df_escola))
    y_vals = densidade(x_vals)

    # Plotagem da density area
    ax.fill_between(x_vals, y_vals,
                    color=sns.color_palette('viridis',
                                            as_cmap=True)(i/len(partidos)),
                    alpha=0.3)
    
    # Adiciona o histograma
    sns.histplot(df_escola['porcentagem_votos_validos_2024'], ax=ax, stat="density", color="black",
                 edgecolor="black", fill=True, 
                 bins=15, alpha=0.1)
    ax.set_title(f'{sigla_partido_prefeito_eleito}', fontsize=15)
    ax.set_ylabel('Densidade')
    ax.set_xlabel('porcentagem_votos_validos_2024')

plt.tight_layout()
plt.show()



## Gráfico de desempenho eleitoral x emendas_pix (OLS)

In [ ]:
plt.figure(figsize=(15,10))
sns.regplot(
    data=df,
    x='emendas_pix_per_capita_partido_prefeito_eleito',
    y='porcentagem_votos_validos_2024',
    marker='o',
    ci=False,
    scatter_kws={"color": 'dodgerblue', "alpha": 0.8, "s": 200},
    line_kws={"color": 'grey', "linewidth": 5}
)

# switch to log scale on the x-axis
plt.xscale('linear')

plt.xlabel('Emendas per capta recebida município', fontsize=20)
plt.ylabel('Desempenho eleitoral', fontsize=20)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

## Gráfico de desempenho x emendas_pix (OLS) por partido separadamente



In [ ]:
# Normalização min–max da coluna de emendas (toda a base de uma vez)
min_em = df['emendas_pix_per_capita_partido_prefeito_eleito'].min()
max_em = df['emendas_pix_per_capita_partido_prefeito_eleito'].max()
df['emendas_norm'] = (
    df['emendas_pix_per_capita_partido_prefeito_eleito'] - min_em
) / (max_em - min_em)

# Obtenção da lista de partidos
partidos = df['sigla_partido_prefeito_eleito'].unique()

# Definição do número de cores na paleta viridis
num_cores = len(partidos)
cor_escola = dict(zip(partidos, sns.color_palette('viridis', num_cores)))


for partido in partidos:
    data = df[df['sigla_partido_prefeito_eleito'] == partido]

    # lmplot retornando FacetGrid
    g = sns.lmplot(
        x='emendas_norm',
        y='porcentagem_votos_validos_2024',
        data=data,
        hue='sigla_partido_prefeito_eleito',
        height=6,
        aspect=1.5,
        ci=False,
        palette=[cor_escola[partido]]
    )

    # Títulos e rótulos
    g.set_xlabels("Emendas PIX per capita (normalizado 0–1)", fontsize=20)
    g.set_ylabels("Desempenho eleitoral", fontsize=20)
    plt.title(f"Desempenho eleitoral - Partido {partido}", fontsize=20)

    # Ajuste de ticks na escala 0–1
    xticks = np.linspace(0, 1, 11)
    g.ax.set_xticks(xticks)
    g.ax.set_xticklabels([f"{x:.1f}" for x in xticks], fontsize=14)
    g.ax.tick_params(axis='y', labelsize=14)

    plt.tight_layout()
    plt.show()

    time.sleep(1)

## Gráfico de desempenho eleitoral em função da variável emendas_pix_per_capta

In [ ]:

# Variação entre estudantes de uma mesma escola e entre escolas diferentes
# Visualização do contexto!
# NOTE QUE A PERSPECTIVA MULTINÍVEL NATURALMENTE CONSIDERA O COMPORTAMENTO
#HETEROCEDÁSTICO NOS DADOS!

palette = sns.color_palette('viridis',
                            len(df['sigla_partido_prefeito_eleito'].unique()))

plt.figure(figsize=(20,14))
sns.scatterplot(data=df, x='emendas_pix_per_capita_partido_prefeito_eleito', y='porcentagem_votos_validos_2024', hue='sigla_partido_prefeito_eleito',
                palette=palette, s=200, alpha=0.8, edgecolor='w')

for partido in df['sigla_partido_prefeito_eleito'].cat.categories:
    subset = df[df['sigla_partido_prefeito_eleito'] == partido]
    sns.regplot(data=subset, x='emendas_pix_per_capita_partido_prefeito_eleito', y='porcentagem_votos_validos_2024', scatter=False, ci=False,
                line_kws={"color": palette[df['sigla_partido_prefeito_eleito'].cat.categories.get_loc(partido)], 'linewidth': 4})

plt.xlabel('Emendax pix per capta', fontsize=20)
plt.ylabel('Desempenho eleitoral', fontsize=20)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(title='Município', title_fontsize='10', fontsize='10', loc='upper left')
plt.show()



In [ ]:

##############################################################################
#                        ESTIMAÇÃO DO MODELO NULO HLM2                       #
##############################################################################

# Estimação do modelo nulo (função 'MixedLM' do pacote 'statsmodels')

modelo_nulo_hlm2 = sm.MixedLM.from_formula(formula='porcentagem_votos_validos_2024 ~ 1',
                                           groups='sigla_partido_prefeito_eleito',
                                           re_formula='1',
                                           data=df).fit()

# Parâmetros do 'modelo_nulo_hlm2'
modelo_nulo_hlm2.summary()



In [ ]:
import statsmodels.formula.api as smf

In [ ]:
# Modelo nulo
null_model = smf.mixedlm(
    'porcentagem_votos_validos_2024 ~ 1',
    df,
    groups=df['sigla_partido_prefeito_eleito'],
    vc_formula={'estado': '0 + C(sigla_municipio)'}
)
null_res = null_model.fit()
print(null_res.summary())


In [ ]:

#intercepto

teste = float(modelo_nulo_hlm2.cov_re.iloc[0, 0]) /\
    float(pd.DataFrame(modelo_nulo_hlm2.summary().tables[1]).iloc[1, 1])

p_value = 2 * (1 - stats.norm.cdf(abs(teste)))

print(f"Estatística z para a Significância dos Efeitos Aleatórios: {teste:.3f}")
print(f"P-valor: {p_value:.3f}")

if p_value >= 0.05:
    print("Ausência de significância estatística dos efeitos aleatórios ao nível de confiança de 95%.")
else:
    print("Efeitos aleatórios contextuais significantes ao nível de confiança de 95%.")


In [ ]:

##############################################################################
#                   COMPARAÇÃO DO HLM2 NULO COM UM OLS NULO                  #
##############################################################################

# Estimação de um modelo OLS nulo

modelo_ols_nulo = sm.OLS.from_formula(formula='porcentagem_votos_validos_2024 ~ 1',
                                      data=df).fit()

# Parâmetros do 'modelo_ols_nulo'
modelo_ols_nulo.summary()



In [ ]:

#até o momento

df_llf = pd.DataFrame({'modelo':['OLS Nulo','HLM2 Nulo'],
                      'loglik':[modelo_ols_nulo.llf,modelo_nulo_hlm2.llf]})

fig, ax = plt.subplots(figsize=(15,15))

c = ['dimgray','darkslategray']

ax1 = ax.barh(df_llf.modelo,df_llf.loglik, color = c)
ax.bar_label(ax1, label_type='center', color='white', fontsize=40)
ax.set_ylabel("Modelo Proposto", fontsize=24)
ax.set_xlabel("LogLik", fontsize=24)
ax.tick_params(axis='y', labelsize=20)
ax.tick_params(axis='x', labelsize=20)
plt.show()



In [ ]:

#'modelo_ols_nulo'

def lrtest(modelos):
    modelo_1 = modelos[0]
    llk_1 = modelo_1.llf
    llk_2 = modelo_1.llf
    
    if len(modelos)>1:
        llk_1 = modelo_1.llf
        llk_2 = modelos[1].llf
    LR_statistic = -2*(llk_1-llk_2)
    p_val = stats.chi2.sf(LR_statistic, 1) # 1 grau de liberdade
    
    print("Likelihood Ratio Test:")
    print(f"-2.(LL0-LLm): {round(LR_statistic, 2)}")
    print(f"p-value: {p_val:.3f}")
    print("")
    print("==================Result======================== \n")
    if p_val <= 0.05:
        print("H1: Different models, favoring the one with the highest Log-Likelihood")
    else:
        print("H0: Models with log-likelihoods that are not statistically different at 95% confidence level")



In [ ]:

#dos 'modelo_ols_nulo' e 'modelo_nulo_hlm2'

lrtest([modelo_ols_nulo, modelo_nulo_hlm2])



In [ ]:

##############################################################################
#     ESTIMAÇÃO DO MODELO COM INTERCEPTOS E INCLINAÇÕES ALEATÓRIOS HLM2      #
##############################################################################

# Estimação do modelo com interceptos e inclinações aleatórios

modelo_intercept_inclin_hlm2 = sm.MixedLM.from_formula(formula='porcentagem_votos_validos_2024 ~ emendas_pix_per_capita_partido_prefeito_eleito',
                                                       groups='sigla_partido_prefeito_eleito',
                                                       re_formula='emendas_pix_per_capita_partido_prefeito_eleito',
                                                       data=df).fit()

# Parâmetros do 'modelo_intercept_inclin_hlm2'
modelo_intercept_inclin_hlm2.summary()

